# Tranformers

## Building a Transformer from Scratch

In [ ]:
import torch

In [ ]:
model = torch.nn.Transformer()
model.encoder.layers[0]

## Attention Mechanisms

### Dot Product Attention

### Scaled Dot Product Attention

In [ ]:
import numpy as np

In [ ]:
small_dots = [
    np.dot(np.random.randn(10),
           np.random.randn(10))
    for i in range(100)]
np.mean(np.absolute(small_dots))

In [ ]:
large_dots = [np.dot(np.random.randn(10000),
                     np.random.randn(10000))
              for i in range(100)]
np.mean(np.absolute(large_dots))

### Multi Head Self Attention

### Adaptive Attention Span

### Persistent Memory/All-Attention

### Product-Key Memory

## Transformers for Computer Vision

# Fine-tuning BERT for sentiment analysis

## Contents
1. 학습데이터 확인
2. 보조함수 확인
3. SenitmentClassifier 확인
4. Analyser 확인
5. 학습 과정 확인
6. tests

## 학습 데이터

In [ ]:
!pip3 install torch
!pip3 install transformers
from typing import List, Tuple
from transformers import BertTokenizer, BertModel
import torch
from torch.nn import functional as F


DATA: List[Tuple[str, int]] = [
    # 긍정적인 문장 - 1
    ("난 너를 좋아해", 1),
    # --- 부정적인 문장 - 레이블 = 0
    ("난 너를 싫어해", 0)
]

TESTS = [
    "나는 자연어처리가 좋아",
    "나는 자연어처리가 싫어",
    "나는 너가 좋다",
    "너는 참 좋다",
]

## 보조함수

In [ ]:
# cuda를 사용할 수 있는지를 체크, 사용가능하다면 cuda로 설정된 device를 출력.
def load_device() -> torch.device:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return device


# 텐서를 구축하는 부분 - X
def build_X(sents: List[str], tokenizer: BertTokenizer, device: torch.device) -> torch.Tensor:
    """
    return X (N, 3, L).  N, 0, L -> input_ids / N, 1, L -> token_type_ids / N, 2, L -> attention_mask
    """
    encodings = tokenizer(text=sents,
                          add_special_tokens=True,
                          return_tensors='pt',
                          truncation=True,
                          padding=True)
    return torch.stack([
        encodings['input_ids'],
        encodings['token_type_ids'],
        encodings['attention_mask']
    ], dim=1).to(device)

# 텐서를 구축하는 부분 - y
def build_y(labels: List[int], device: torch.device) -> torch.Tensor:
    return torch.FloatTensor(labels).unsqueeze(-1).to(device)

## Sentiment Classifier

In [ ]:
class SentimentClassifier(torch.nn.Module):
    def __init__(self, bert: BertModel):
        super().__init__()
        self.bert = bert
        self.hidden_size = bert.config.hidden_size
        # TODO 1
        self.W_hy = torch.nn.Linear(..., ...)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X: (N, 3, L)
        :return: H_all (N, L, H)
        """
        input_ids = X[:, 0]
        token_type_ids = X[:, 1]
        attention_mask = X[:, 2]
        H_all = self.bert(input_ids, token_type_ids, attention_mask)[0]
        return H_all

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X:
        :return:
        """
        # TODO 2
        H_all = self.forward(X)
        H_cls = ...
        y_hat = ...  # (N, H) * (H, 1) -> (N, 1)
        return torch.sigmoid(y_hat)

    def training_step(self, X: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        # TODO 3
        y_hat = self.predict(X)
        loss = ...
        return loss.sum()

## Analyser

In [ ]:

class Analyser:
    """
    Bert 기반 감성분석기.
    """
    def __init__(self, classifier: SentimentClassifier, tokenizer: BertTokenizer, device: torch.device):
        self.classifier = classifier
        self.tokenizer = tokenizer
        self.device = device

    def __call__(self, text: str) -> float:
        X = build_X(sents=[text], tokenizer=self.tokenizer, device=self.device)
        y_hat = self.classifier.predict(X)
        return y_hat.item()

## Training

In [ ]:
# 사전학습된 버트 모델을 로드
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
bert = BertModel.from_pretrained('beomi/kcbert-base')

# --- have a look at the config --- #
print(bert.config)
print(bert.config.hidden_size)

In [ ]:
# --- hyper parameters --- #
EPOCHS = 20
LR = 0.0001


device = load_device()
print(device)

# --- build the dataset --- # 
sents = [sent for sent, _ in DATA]
labels = [label for _, label in DATA]
X = build_X(sents, tokenizer, device)
y = build_y(labels, device)

# --- instantiate the classifier --- #
classifier = SentimentClassifier(bert)
classifier.to(device)  # 모델도 gpu에 올리기. 
optimizer = torch.optim.Adam(classifier.parameters(), lr=LR)  # 최적화 알고리즘을 선택.

# --- 학습시작 --- #
for epoch in range(EPOCHS):
    loss = classifier.training_step(X, y)
    loss.backward()  # 오차 역전파
    optimizer.step()  # 경사도 하강
    optimizer.zero_grad()  # 기울기 축적방지
    print(f"epoch:{epoch}, loss:{loss.item()} ")

## Test

In [ ]:
classifier.eval()
analyser = Analyser(classifier, tokenizer, device)

for sent in TESTS:
    print(sent, "->", analyser(sent))

## Conclusion